# 0. Импорт библиотек

In [16]:
import pandas as pd

from IPython.display import clear_output
import time
import warnings
warnings.filterwarnings("ignore")

from pybliometrics.scopus import ScopusSearch # extract data
from scidownl import scihub_download # download data
#import fitz # pdf to text

# 1. Парсинг

## 1.1 Таблица кодовых слов Scopus

> Источник, в котором можно посмотреть какие ключ слова можно использовать: [ссылка](https://dev.elsevier.com/sc_search_tips.html)
 
> Документация к библиотеке для парсинга Scopus с помощью SciHub: [ссылка](https://pybliometrics.readthedocs.io/en/stable/)


***Используемые кодовые слова представлены в таблице ниже***

<div align="center">

| Кодовое слово | Описание | переменная в коде |
|:--------------|:---------|:-----------------:|
|**TITLE-ABS-KEY()**|Возвращает статьи с заданной фразой <br/> в ключевых словах, абстракте и заголовке|`KEYS:list`|
|**PUBYEAR()**|Возвращает статьи с заданным годом публикации|`PUBYEAR:list`|
|**DOCTYPE()**|Возвращает статьи с заданным типом публикации.<br/>Мы будем использовать только *'ar'* – Article|Вшита в метод `get_papers()`|
|**SUBJAREA()**|Возвращает статьи с заданной предметной областью.<br/>Мы будем использовать только *'MATE'* – Materials Science|Вшита в метод `get_papers()`|
|**LANGUAGE()**|Возвращает статьи с заданным языком.<br/>Мы будем использовать только *'english'*|Вшита в метод `get_papers()`|
</div>

## 1.2 Список необходимых ключевых слов по которым производится поиск

In [17]:
KEYS = ["MR", "GMR","Magnetic resistance", "Colossal magnetic resistance", 
        "resistivity in magnetic field", "magnetic field", "magnetic properties",
        "conduction in magnetic field"]
PUBYEAR = [i for i in range(1970,2021)]

## 1.3 Написание запросов

In [18]:
def get_papers(keys:list, pubyear:list, doctype:str = 'ar', subjarea:str = 'MATE', language:str = 'english') -> pd.DataFrame:
    
    """ Method for extracting science papers from scopus by given keywords and years of publish \n
            
                Method parameters:
                
                    keys : list
                        Keywords. Are set in a list, the method is iterated for each keyword from a given list  [Watch more](https://dev.elsevier.com/sc_search_tips.html#:~:text=in%20their%20title.-,TITLE-ABS-KEY,-A%20combined%20field)
                    pubyear : list
                        Years of publications. Are set in a list, the method is iterated for each keyword from a given list [Watch more](https://dev.elsevier.com/sc_search_tips.html#:~:text=Year%20of%20Publication)
                    doctype : str, default 'ar'
                        Type of document. As default, it is 'ar' – article; [Watch more](https://dev.elsevier.com/sc_search_tips.html#:~:text=DOCTYPE(XX))
                    subjarea : str, default 'MATE'
                        Subject Area.  As default, it is 'MATE' – Materials Science [Watch more](https://dev.elsevier.com/sc_search_tips.html#:~:text=from%20journal%20sources.-,SUBJAREA,-(XX))
                    language : str, default 'english'
                        language of paper. As default, it is 'english' [Watch more](https://dev.elsevier.com/sc_search_tips.html#:~:text=is%20a%20keyword.-,LANGUAGE,-%40)
        
                Returns:
            
                    pd.DataFrame:dataframe – DataFrame with all information from Scopus
    """
    
    dataframe = pd.DataFrame()


    for key in keys:
        clear_output(wait=False)
        print('Progress scanning keywords:')
        print(f'Keyword: {key}')

        for year in pubyear:
            print(f'Year: {year}')

            query = f'(TITLE-ABS-KEY({key}) AND DOCTYPE({doctype}) AND SUBJAREA({subjarea}) AND PUBYEAR = {year} AND LANGUAGE({language}))'
            try:
                data = ScopusSearch(query, subscriber=False, verbose=False)
                data_q = pd.DataFrame(pd.DataFrame(data.results))            

                data_q[['keyword', 'doctype', 'subjarea', 'year']] = key, doctype, subjarea, year
            
                dataframe = pd.concat([dataframe, data_q])
            except:
                continue
    return dataframe


In [19]:
#df = get_papers(KEYS, PUBYEAR)
#df.to_csv('downloaded_articles.csv')

Данные были получены и выгружены в отдельный csv файл

In [20]:
df = pd.read_csv('downloaded_articles.csv')

# 2 Предобработка

In [12]:
df = df.dropna(subset='doi')

In [16]:
df = df.drop(['pii','pubmed_id', 'creator', 'afid', 'author_count', 'author_names','author_ids', 'author_ids'], axis=1)

KeyError: "['pii', 'pubmed_id', 'creator', 'afid', 'author_count', 'author_names', 'author_ids', 'author_ids'] not found in axis"

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177191 entries, 0 to 187203
Data columns (total 34 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           177191 non-null  int64  
 1   eid                  177191 non-null  object 
 2   doi                  177191 non-null  object 
 3   title                177191 non-null  object 
 4   subtype              177191 non-null  object 
 5   subtypeDescription   177191 non-null  object 
 6   affilname            174921 non-null  object 
 7   affiliation_city     174214 non-null  object 
 8   affiliation_country  174786 non-null  object 
 9   author_afids         0 non-null       float64
 10  coverDate            177191 non-null  object 
 11  coverDisplayDate     177191 non-null  object 
 12  publicationName      177187 non-null  object 
 13  issn                 175736 non-null  object 
 14  source_id            177187 non-null  float64
 15  eIssn                5

# Downloading

In [32]:
for i, doi in enumerate(df['doi'].sample(4)):
    paper_type = "doi"
    url = 'https://sci-hub.ru/' + doi
    out = "./papers/paper_" + doi[8::] + ".pdf"  # output file path
    df.loc[i,'file_path'] = out
    #print(df.loc[i,'file_path'])
    #print(doi)
    proxies = {"http": "socks5://127.0.0.1:7890"}  # optional proxies
    time.sleep(10)
    scihub_download(url, paper_type=paper_type, out=out, proxies=proxies)
    clear_output(wait=False)

Оцифровка статей будет проводиться через нейронную сеть Nougat. [Ссылка](https://github.com/facebookresearch/nougat/tree/main)
 
Видео с применением этой нейронки в коллабе. [Ссылка](https://www.youtube.com/watch?v=SYO_4uhdHKM)

SyntaxError: unexpected character after line continuation character (3091123736.py, line 1)